# 3. Allocations

In [139]:
import pandas as pd
import numpy as np
from plotnine import ggplot, aes, geom_bar, labs

In [140]:
file_path = "./data/multi_asset_etf_data.xlsx"

In [141]:
excess_returns_df = pd.read_excel(file_path, 
                                  sheet_name="excess returns", 
                                  parse_dates=["Date"],
                                  index_col="Date") # monthly excess returns

excess_returns_df = excess_returns_df.drop(columns=["QAI"]) #resolve issue with data

In [142]:
MU_TARGET = 0.01

In [143]:
def calc_tan_weights(returns):
    """
    Calculate tangency weights for a set of returns
    """
    cov_matrix = returns.cov() 
    cov_vals = cov_matrix.values
    mean_vals = returns.mean().values

    w_unnormalized = np.linalg.inv(cov_vals) @ mean_vals
    w_tan = w_unnormalized / np.sum(w_unnormalized)

    return w_tan

# Comparison

In [144]:
assets = excess_returns_df.columns
num_assets = len(assets)
weights = pd.DataFrame(index=assets)

weights['Equal Weights'] = 1/num_assets #Equally Weighted
weights['Risk Parity Weights'] = 1 / (excess_returns_df.var()) #Risk Parity Weighted
weights['Mean-Variance Weights'] = calc_tan_weights(excess_returns_df) #Mean-Variance Weighted

In [145]:
unscaled_portfolio_mean_returns = (excess_returns_df.mean() @ weights)
unscaled_portfolio_mean_returns.round(4)

Equal Weights             0.0038
Risk Parity Weights      32.3871
Mean-Variance Weights     0.0107
dtype: float64

In [146]:
scaled_weights = weights * MU_TARGET/unscaled_portfolio_mean_returns

In [147]:
# test
excess_returns_df * scaled_weights["Equal Weights"]

,BWX,DBC,EEM,EFA,HYG,IEF,IYR,PSP,SPY,TIP
Date,,,,,,,,,,
2011-02-28,0.001865,0.011182,-0.000007,0.009533,0.003919,-0.000445,0.012110,0.010767,0.009331,0.002022
2011-03-31,0.002324,0.007144,0.016785,-0.006253,0.000200,-0.000323,-0.002816,0.004512,0.000119,0.003247
2011-04-30,0.012945,0.012083,0.007243,0.014924,0.004230,0.004877,0.012369,0.015565,0.007677,0.006301
2011-05-31,-0.003437,-0.013573,-0.007646,-0.005698,0.000513,0.006811,0.002850,-0.010876,-0.002818,0.000865
2011-06-30,0.000061,-0.011235,-0.002397,-0.003128,-0.001428,-0.001252,-0.008159,-0.011218,-0.004392,0.002091
...,...,...,...,...,...,...,...,...,...,...
2025-01-31,-0.000583,0.006495,0.004882,0.011914,0.002780,0.000805,0.004189,0.017220,0.006298,0.002806
2025-02-28,0.003020,-0.000332,0.002351,0.007146,0.001882,0.006739,0.009357,-0.011342,-0.004065,0.005033
2025-03-31,0.001994,0.005279,0.002256,-0.000267,-0.003637,0.000153,-0.006962,-0.016903,-0.015547,0.001050


In [148]:
def performance_statistics(excess_returns: pd.DataFrame, weights: pd.Series, label: str):
    """
    Calculate performance of portfolios
    """
    portfolio_performance = ((excess_returns * weights[label]).sum(axis = 1) + 1).prod()

    monthly_mean = excess_returns.mean() @ weights[label]
    monthly_vol = weights[label] @ excess_returns.corr() @ weights[label]
    annualized_mean = monthly_mean * 12
    annualized_vol = monthly_vol * np.sqrt(12)
    annualized_sharpe = (monthly_mean / monthly_vol) * np.sqrt(12)

    portfolio_performance_statistics = pd.DataFrame({
        "Portfolio Type": [label],
        #"Performance_over_Sample": [portfolio_performance],
        "Annualized Mean": [annualized_mean],
        "Annualized Volatility": [annualized_vol],
        "Annualized Sharpe Ratio": [annualized_sharpe]
    })

    return portfolio_performance_statistics

In [149]:
eq_performance = performance_statistics(excess_returns_df, scaled_weights, "Equal Weights")
rp_performance = performance_statistics(excess_returns_df, scaled_weights, "Risk Parity Weights")
mv_performance = performance_statistics(excess_returns_df, scaled_weights, "Mean-Variance Weights")

In [150]:
comparison = pd.concat(
        [eq_performance,
        rp_performance,
        mv_performance]
    )
comparison.round(4)


,Portfolio Type,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
0,Equal Weights,0.12,13.5732,0.0088
0,Risk Parity Weights,0.12,39.1237,0.0031
0,Mean-Variance Weights,0.12,2.8077,0.0427
